# Compute travel time of reflected waves

In this notebook, we derive the pS and sP travel time for the 4-meter-long biaxial apparatus with gouge patch source.

<img src="compute_reflection_schematic.png" alt="schematic" width="600"/>


We numerically solve simultaneous equations of the symmetry of reflection (isosceles triangle by $\vec{m}$ and $\vec{w}$) and the Snell's law.

$$ ||\vec{m}|| \cos{\theta_2} = \dfrac{1}{2}||\vec{w}|| $$
$$ \dfrac{\sin{\theta_1}}{c_{s/p}} = \dfrac{\sin{\theta_2}}{c_{p/s}} $$ 

We solve these equations to identify the optimized incident and reflected angles for the pS and sP waves. The travel times are computed by law of cosines as follows.

$$ l_1 = ||\vec{r}||^2 + ||\vec{m}||^2 - \vec{r} \cdot \vec{m} $$ 
$$ l_2 = ||\vec{m}||^2 + ||\vec{w}||^2 - \vec{m} \cdot \vec{w} $$

$$ t_{sP/pS} = l_1/c_{s/p} + l_2/c_{p/s} $$

# Function to compute travel time

```python
def compute_ps_and_sp_side_reflection2(tr_obs, param):
    """
    compute ps and sp reflection from side surface by solving the simultaneous equation with Snell's law 
    """
    cp = param["cp"] # P wave velocity [m/s]
    cs = param["cs"] # S wave velocity[m/s]

    rock_width = 0.1 #[m] 100mm width of fault
    rock_hight = 0.2 #[m] 200mm on onside of rock specimen
    sensorloc = np.array([0, 0, 70e-3]) # location of sensor
    sourceloc = np.array([tr_obs.stats.xi1, tr_obs.stats.eta1, tr_obs.stats.zeta1]) # location of source

    rvec = np.array(sourceloc - sensorloc) # vector from sensor to source
    wvec_side = np.array([0, -2*rock_width, 0]) # vector from sensor to the virtual side of sensor for reflection

    # Function to compute pS
    vnorm = np.linalg.norm

    def func_res(x, type):
        """
        Define the simultaneous equations associated with the symmetry of reflection (isosceles triangle by mvec and wvec)
        and the Snell's law 
        """
        p, q = x # parameters to define the reflection point by mvec
        if p<0 or q<0:
            return [np.inf, np.inf] 

        mvec = p*rvec + q*wvec_side # vector from sensor to reflection point
        svec = mvec - rvec # vector from source to the reflection point
        cos_i = np.dot(svec, wvec_side)/(vnorm(svec) * vnorm(wvec_side)) # incident angle from source to vector 
        cos_j = np.dot(mvec, wvec_side)/(vnorm(mvec) * vnorm(wvec_side)) # reflected angle from source to vector 
        sin_i = np.sqrt(1-cos_i**2)
        sin_j = np.sqrt(1-cos_j**2)
        f = np.zeros(2)
        f[0] = np.dot(mvec, wvec_side) / vnorm(wvec_side)**2 - 0.5 # condition of isoscales triangle from sensor to the virtual side of sensor.
        if type=="pS":
            f[1] = sin_i/cp - sin_j/cs # Snell's law
        elif type=="sP":
            f[1] = sin_i/cs - sin_j/cp # Snell's law
        else:
            raise ValueError(f"type {type} is unknown.")
            
        return f

    tt_dict = dict()
    for type in ["pS", "sP"]:
        A = fsolve(func_res, [0.1, 0.1], args=(type,), full_output=True) # compute reflection point
        assert A[2] # check if the solution is found
        # print(A)
        p0, q0 = A[0]
        mvec_side = p0*rvec + q0*wvec_side # vector from sensor to the optimized reflection point
        svec_side = mvec_side - rvec # vector from source to the optimized reflection point
        ideg = np.rad2deg(np.arccos(np.dot(svec_side, wvec_side)/(vnorm(svec_side) * vnorm(wvec_side)))) # optimized incident angle
        jdeg = np.rad2deg(np.arccos(np.dot(mvec_side, wvec_side)/(vnorm(mvec_side) * vnorm(wvec_side)))) # optimized reflected angle

        l1 = np.sqrt( np.linalg.norm(rvec)**2 + np.linalg.norm(mvec_side)**2 - 2 * np.dot(rvec, mvec_side) ) # distance from source to reflection point
        l2 = np.sqrt( np.linalg.norm(mvec_side)**2 + np.linalg.norm(wvec_side)**2 - 2 * np.dot(mvec_side, wvec_side) ) # distance from reflection point to sensor
        
        if type=="pS":
            assert (ideg - jdeg) > 1e-8 # check if incident angle is larger than reflected angle
            tt_dict[type] = l1/cp + l2/cs # compute travel time as summation of incident and reflected parts

        elif type=="sP":
            assert (ideg - jdeg) < -1e-8 # check if incident angle is smaller than reflected angle
            tt_dict[type] = l1/cs + l2/cp # compute travel time as summation of incident and reflected parts

        else:
            raise ValueError(f"type {type} is unknown.")
    
        tt_dict[type+"_angles"] = (ideg, jdeg) # store incident and reflected angles

    return tt_dict["pS"], tt_dict["sP"], tt_dict["pS_angles"], tt_dict["sP_angles"]

```